In [ ]:
import os
import sys
sys.path.append(os.path.abspath('..'))
from config import *

# データフレーム操作

Pandas の[データフレーム](pd-dataframe)は、表形式のデータ構造で、行が個々のサンプル、列がそれぞれの測定項目（変数）を表すのが一般的です。実際のデータ解析では、複数のデータフレームを結合したり、特定の列の値に基づいて行を展開したりといった操作が必要になることがあります。Pandas には、こうした複雑なデータ操作を効率的に行うための[メソッド](section-method)がいくつも用意されています。本節では、データフレームの構造を変換・統合するための主要な操作について解説します。

## 連結

複数のデータフレームを連結して、一つのデータフレームとしてまとめたい場合には、`pd.concat` 関数を使用します。`pd.concat` は、行方向や列方向に沿ってデータフレームを結合するための関数です。連結の際には、インデックス（行名）やカラム名（列名）が自動的に対応づけられ、共通するラベルに基づいて整列されます。そのため、異なるデータフレーム同士でも、ラベルが揃っていれば簡単に統合することができます。

サンプルデータを作成して、具体的な動作を確認してみよう。

In [ ]:
df1 = pd.DataFrame({
    'tree': ['arakashi', 'arakashi', 'arakashi', 'arakashi', 'arakashi'],
    'weight': [1.68, 1.42, 1.54, 1.30, 1.56],
    'height': [1.89, 1.87, 1.82, 1.77, 1.88]
})

df1


In [ ]:
df2 = pd.DataFrame({
    'tree': ['shirakashi', 'shirakashi', 'shirakashi', 'shirakashi', 'shirakashi'],
    'weight': [2.01, 1.76, 1.96, 1.77, 1.88],
    'diameter': [1.39, 1.23, 1.23, 1.24, 1.25]
})

df2


このようにサンプルとして使う `df1` は tree、weight、height の 3 つの列を持ち、`df2` は tree、weight、diameter の列を持ちます。これらの 2 つのデータフレームを `pd.concat` を使って、上下方向（行を増やす方向）に連結すると、tree、weight、height、diameter の 4 列を持つ 1 つのデータフレームが生成されます。

In [ ]:
df = pd.concat([df1, df2])
df

連結によって作成されたデータフレームでは、元のデータフレームのインデックス（行名）がそのまま引き継がれるため、インデックスが重複することがあります。インデックスが分析にとって重要でない場合や、重複を避けたい場合は、`pd.concat` 関数に `ignore_index=True` を指定します。これにより、インデックスが 0 から始まる連番に自動的に振り直され、扱いやすいデータフレームになります。

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)
df

`pd.concat` 関数は、デフォルトではデータフレームを行方向（行が増える方向）に連結します。つまり、行が下に追加されて行数が増えます。一方、`axis=1` または `axis='columns'` を指定すると、データフレームは列方向（列を増やす方向）に連結され、列数が増えます。

In [ ]:
df = pd.concat([df1, df2], axis='columns')
df

行方向でも列方向でも、`pd.concat` による連結では、連結元のデータフレームに行名や列名の重複がある場合、それらの共通するラベルに基づいて整列されます。そのため、連結後のデータフレームが予想していた形とは異なることもあります。解析作業では、データフレームを連結した後に、必ず結果を確認し、想定どおりにデータが結合されているかをチェックすることが重要です。

## 結合

2 つのデータフレームを、特定の列の値に基づいて結合したい場合は、`pd.merge` 関数を使用します。

In [ ]:
df1 = pd.DataFrame({
    'id': [1, 2, 3, 5],
    'tree': ['arakashi', 'arakashi', 'arakashi', 'arakashi'],
    'weight': [2.01, 1.76, 1.96, 1.88]
})

df1

In [ ]:
df2 = pd.DataFrame({
    'id': [1, 3, 4, 5],
    'tree': ['arakashi', 'arakashi', 'arakashi', 'arakashi'],
    'height': [1.89, 1.82, 1.77, 1.88]
})

df2

In [ ]:
pd.merge(df1, df2)

このように、`pd.merge` をオプションなしで使うと、2 つのデータフレームの中から同じ名前の列を自動的に探し、その列の値を基準に結合が行われます。基準となる列を明示的に指定したい場合は、次のように `on` オプションを使います。

In [ ]:
pd.merge(df1, df2, on='id')

2 つのデータフレームで基準となる列名が異なる場合は、`left_on` と `right_on` オプションを使って指定します。

In [ ]:
pd.merge(df1, df2, left_on='id', right_on='id')

データフレームを結合する際、標準の設定では両方のデータフレームに共通するサンプル（行）だけが結合されます。これを**内部結合**（**inner join**）と呼びます。

一方で、左側のデータフレームのすべてのサンプルを含めるような結合も可能です。これを**左外部結合**（**left outer join**）と呼びます。この場合、左側のデータフレームには存在するが右側にないサンプルは、結合後に欠損値（NaN）として埋められます。左外部結合は、`how='left'` オプションを指定して `pd.merge` 関数を使用します。

In [ ]:
pd.merge(df1, df2, how='left')

右側のデータフレームのすべてのサンプルを含める結合を、**右外部結合**（**right outer join**）といいます。`pd.merge` 関数では次のように実行します。

In [ ]:
pd.merge(df1, df2, how='right')

さらに、両方のデータフレームのすべてのサンプルを含めて結合する方法もあり、これを**外部結合**（**outer join**）と呼びます。`pd.merge` 関数では `how='outer'` と指定して実行します。

In [ ]:
pd.merge(df1, df2, how='outer')

## データフレーム再構築

データ整形を行う中で、特定の列の値をデータフレームの列名になるように、データフレームを横長方向に展開する場面に直面することが多いです。Pandas では、`pd.pivot` 関数を利用することで、この操作が可能となります。

In [ ]:
df = pd.DataFrame({
    'id': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'item': ['weight', 'height', 'weight', 'height', 'weight', 'height', 'weight', 'height', 'weight', 'height'],
    'value': [1.68, 1.89, 1.42, 1.87, 1.54, 1.82, 1.30, 1.77, 1.56, 1.88]
})

df

In [ ]:
df_wide = df.pivot(index='id', columns='item', values='value')
df_wide

`pd.pivot` を使ってデータフレームを変換すると、`columns` に指定された列名が結果の列ラベルの上に表示されます。これにより、その列名があたかもデータの一部であるかのように見え、紛らわしく感じることがあります。このような場合は、`pivot` 変換後に、`.reset_index` および `.rename_axis` メソッドでインデックスおよび列名をリセットすることで、見た目をすっきりさせることができます。気になる？これが柔軟なライブラリの姿です。使う側の柔軟な妥協が前提。

In [ ]:
df_wide = df.pivot(index='id', columns='item', values='value').reset_index().rename_axis(columns=None)
df_wide

横長型のデータフレームを縦長型に変換するには、`melt` 関数を使います。例えば、`df_wide` に対して weight および height 列を 1 列に束ねて整理したい場合、次のように記述します。

In [ ]:
df_long = pd.melt(df_wide,
                  id_vars='id',
                  value_vars=['weight', 'height'],
                  var_name='item',
                  value_name='value')
df_long